# Entrega 2 - Predictor de muerte de pacientes con HIV utilizando Naive Bayes

### Grupo 26:
     - A. Martínez
     - J. Mezquita
     - N. Núñez

## 1. Objetivo

El objetivo principal de esta tarea es construir un algoritmo capaz de predecir la muerte de pacientes bajo observación por HIV utilizando metodos bayesianos, croncretamente el algoritmo de Naive Bayes visto en el curso. Para ello, utilizaremos el dataset `AIDS Clinical Trials Group Study 175` tomando como valor a predecir el indicador de censura `cid`.

El indicador de censura puede tomar dos valores "censoring" o "failure". La censura ocurre cuando un valor de una observación solo se conoce parcialmente. 

En este caso ocurre cuando el experimento termina en un momento determinado, tras el cuál los pacientes todavía vivos quedan todos censurados por la derecha.  

Luego del momento de la última observación no se conocen datos sobre la muerte o supervivencia de los pacientes. Por lo que se puede tomar únicamente el caso "failure" como la muerte de un paciente.


In [1]:
import funciones
import pandas as pd
import numpy as np
import sklearn.preprocessing as sk_pre
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold
from scipy.stats import chi2_contingency
from sklearn.preprocessing import KBinsDiscretizer
import importlib

importlib.reload(funciones)

# Objetivo a predecir
OBJETIVO = 'cid'

dataset = pd.read_csv('data.csv')

## 2. Diseño del predictor

 ### 2.1 Partición del conjunto de datos

Para realizar el entrenamiento, ajuste de la solución y evaluación del modelo, el dataset es separado en 2 conjuntos:

- Entrenamiento (85%): Utilizado para entrenar el algoritmo de Naive Bayes
- Evaluación (15%): Para obtener métricas de rendimiento del modelo una vez finalizado.

Además, para ajustar los hiperparametros del modelo, utilizaremos `validación cruzada`, dividiendo el conjunto de  entrenamiento en 5 partes iguales. Es por esto que no se utiliza una partición de validación.

Pot último, a la hora de separarlos, utilizamos el parámetro `stratify=Y` de forma que la distribución de la columna `cid` sea similar en los 2 subconjuntos.

Al momento de evaluar las soluciones se utilizan las siguientes métricas, calculadas con `scikit-learn`:

- Accuracy: $ \frac{TP + TN}{TP + TN + FP + FN}$
- Precision: $ \frac{TP}{TP + FP}$
- Recall: $ \frac{TP}{TP + FN}$
- F1: $ \frac{2 \cdot Precision}{Precision + Recall} $

Estas métricas serán acompañadas con la visualización de la matriz de confusión y curva precision-recall.

In [2]:
X = dataset.copy().drop(columns=[OBJETIVO, 'pidnum'])
Y = dataset[OBJETIVO].copy()

X_train, X_val, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 12345, stratify=Y)
#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 12345, stratify=Y_train)

### 2.2 Valores posibles

El modelo Naive Bayes necesita conocer de antemano los valores posibles que puede tomar cada atributo, por lo que se construye un diccionario `valores_posibles` que mapee el nombre de cada atributo a los posibles valores que este pueda tomar. 
Esto se hace sobre el dataset original, y no sobre el subconjunto de entrenamiento, puesto que podría ocurrir que, al hacer la partición de estos, el subconjunto de validación/test contenga valores en ciertos atributos que no esten en el subconjunto de entrenamiento.
Además, como este procedimiento se hace sobre el dataset sin ningún preprocesamiento, el modelo sustituirá los valores de las claves pertenecientes al array `atributos_a_categorizar` por sus valores posibles luego de ser categorizados.

In [3]:
valores_posibles = {}

for categoria in X.columns:
    valores_posibles[categoria] = X[categoria].unique()

### 2.3 Preprocesamiento de datos

Debido a que este dataset cuenta con algunos atributos numéricos continuos, se preprocesaran usando la libreria de `scikit-learn`. La categorizacion se hara especificamente con la funcion `KBinsDiscretizer` con los siguientes parametros:
- n_bins   =  3 
- encode   = 'ordinal'
- strategy = 'kmeans'
En la seccion de experimentacion explicaremos el porque de estos parametros.

Los atributos a categorizar son:

- time
- age
- wtkg
- karnof
- preanti
- cd40
- cd420
- cd80
- cd820

Además, el dataset posee una columna `pidnum` que es el identificador de cada observación. Al ser esto un metadato y no un dato de la realidad que influya en el resultado, esta columna se elimina del dataset, para evitar agregar ruído al modelo.

In [4]:
atributos_a_categorizar = ['time', 'age', 'wtkg', 'karnof', 'preanti', 'cd40', 'cd420', 'cd80', 'cd820']

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, encode="ordinal", strategy='kmeans', subsample=200_000, random_state=12345)
puntos_corte = {}
for atributo in atributos_a_categorizar:
    X_train[atributo] = discretizer.fit_transform(X_train[[atributo]]).astype(int)
    puntos_corte[atributo] = discretizer.bin_edges_[0][1:3]
    valores_posibles[atributo] = np.unique(X_train[atributo])

# TODO: cambiar (está mal)
for atributo in atributos_a_categorizar:
    X_val[atributo] = discretizer.fit_transform(X_val[[atributo]]).astype(int)

for atributo in atributos_a_categorizar:
    X_test[atributo] = discretizer.fit_transform(X_test[[atributo]]).astype(int)

### 2.4 Linea base 

Al estar trabajando con el mismo dataset que el informe anterior, sabemos que la linea base sera la misma. Recordemos que dicho dataset cuenta con 1618 entradas cuyo resultado es 0 y 521 cuyo resultado es 1. Por lo que el predictor simple devolverá que el resultado es siempre 0. Los resultados de este predictor eran:

- Accuracy de linea base: 0.7564282374941561
- Precision de linea base: 0.7564282374941561
- Recall de linea base: 1.0
- F1 de linea base: 0.8613255256853873


### 2.5 Selección de atributos

Por último, se aplica también selección de atributos sobre el dataset en aras de intentar obtener mejores resultados.
Para esto aplicamos el método $ \chi^2$ que evalúa la independencia entre las categorias y la categoría objetivo, utilizando un umbral de p = 0.01, es decir, las variables tales que su p-valor sea menor o igual que 0.01, se consideran relevantes. Luego de esto, se iterará sobre todos los subconjuntos del array de atributos tales que su p-valor sea mayor a 0.01, y se seleccionará como atributo a eliminar del dataset aquel subconjunto que de mejores resultados.
A continuación se desarrollarán los resultados del método $ \chi^2$ y la iteración sobre los subconjuntos se hará durante la sección de experimentación.

In [ ]:
correlacion = []

for atributo in X.columns:
    tabla_contingencia = pd.crosstab(X[atributo], Y)
    _, p, _, _ = chi2_contingency(tabla_contingencia)
    correlacion.append((atributo, p))

df_correlacion = pd.DataFrame(correlacion, columns=['Atributo', 'p'])
print("Tabla de correlación:")
print(df_correlacion.to_string(index=False))

# Para auementar la cantidad de potenciales atributos a dropear, se puede aumentar el valor de p
potenciales_atributos_a_dropear = df_correlacion[df_correlacion['p'] > 0.01]['Atributo'].tolist()
print(f'\nAtributos a dropear: {potenciales_atributos_a_dropear}')

### 2.6 Otras decisiones



## 3. Experimentación

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate

m = 0
modeloCV = funciones.NaiveBayesAIDS(m, valores_posibles)

# Definir las métricas con pos_label=0 donde sea necesario
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=0),
    'recall': make_scorer(recall_score, pos_label=0),
    'f1': make_scorer(f1_score, pos_label=0)
}

# Usar cross_validate con las métricas personalizadas
scores = cross_validate(modeloCV, X_train, Y_train, cv=5, scoring=scoring)

# Mostrar los resultados
print("Accuracy:", scores['test_accuracy'].mean())
print("Precision:", scores['test_precision'].mean())
print("Recall:", scores['test_recall'].mean())
print("F1:", scores['test_f1'].mean())


In [ ]:
for m in c:
    print(m)
    
    pipeline = Pipeline([
        ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')),  
        ('actualizar_valores', ActualizarValoresPosiblesTransformer(atributos_a_categorizar, valores_posibles)),  
        ('naive_bayes', funciones.NaiveBayesAIDS(m, valores_posibles))
    ])
    
    # Realizar la validación cruzada
    scores = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=scoring)
    
    # Guardar los resultados
    resultados[m] = [scores['test_accuracy'].mean(), 
                     scores['test_precision'].mean(), 
                     scores['test_recall'].mean(), 
                     scores['test_f1'].mean()]


In [ ]:
clave_max = max(resultados, key=lambda k: resultados[k][0])
print(clave_max)
print(resultados[clave_max])

In [ ]:
funciones.plot_metricas(resultados,1000)

In [ ]:
funciones.plot_metricas(resultados,100)

### 3.2 Selección de atributos

TODO

In [ ]:
atributos_a_eliminar = funciones.seleccionar_subconjunto_a_eliminar(X_train, Y_train, 36, potenciales_atributos_a_dropear, valores_posibles, atributos_a_categorizar, X_validacion, Y_validacion)

### 3.3 Análisis de resultados

La siguiente tabla resume los mejores resultados obtenidos dependiendo de cuando se realizó la categorización y del valor de `max_range_split`, indicando que función de elección de atributo fue la que generó dicho valor.

En general, no se observa un claro superior entre categorizar antes o después de ejecutar ID3, sin embargo, si nos enfocamos únicamente en el accuracy y f1, podemos identificar que los mejores resultados se obtienen utilizando:

- La función gain ratio.
- max-range-split = 3.
- Categorizando los atributos necesarios durante la ejecución del algoritmo ID3.

Si probamos estas características sobre el conjunto de testeo, obtenemos la siguiente matriz de confusión.

In [ ]:
Y_predicho_test = ArbolDecisionMRS3_gain_ratio.predecir(X_test)
funciones.plot_confusion_matrix(Y_test, Y_predicho_test)
accuracy_test, precision_test, recall_test, f1_test = funciones.get_accuracy_precision_recall_f1(Y_test, Y_predicho_test)
print('Accuracy:', accuracy_test)
print('Precision:', precision_test)
print('Recall:', recall_test)
print('F1:', f1_test)

## 4. Comparación

En esta sección se compara el algoritmo ID3 contra la línea base planteada al inicio del documento, además de las implementaciones de la librería scikit-learn:

- DecisionTreeClassifier
- RandomForestClassifier

### 4.1 Preprocesamiento de datos

Para realizar estas comparaciones con la librería `scikit-learn`, primero se preprocesaran los datos ya que estos algoritmos trabajan con atributos numéricos, contrario a la implementación de `ID3` presentada anteriormente. 

Los atributos categóricos `trt` y `strat` utilizan más de 2 categorías en un solo atributo, por lo que estas dos columnas impondrán un orden donde no lo hay, empeorando así el rendimiento de las implementaciones de la librería `scikit-learn`, por lo que se aplica `one-hot encoding` sobre los dos atributos mencionados esperando una mejora en los resultados.

Por otro lado, se observa que en el conjunto de datos no hay elementos faltantes para ningún atributo, por lo que no es necesario realizar preprocesamiento

In [ ]:
X_librerias = dataset.copy().drop(columns=[OBJETIVO, 'pidnum'])
Y_librerias = dataset[OBJETIVO].copy()

X_train_librerias, X_test_librerias, Y_train_librerias, Y_test_librerias = train_test_split(X_librerias, Y_librerias, test_size = 0.15, random_state = 12345, stratify=Y_librerias)
X_train_librerias, X_test_librerias = funciones.aplicar_ohe(dataset, X_train_librerias, X_test_librerias, 'trt')
X_train_librerias, X_test_librerias = funciones.aplicar_ohe(dataset, X_train_librerias, X_test_librerias, 'strat')


Y_predicho_predictor_simple = [0 for _ in range(len(dataset))]
Y_real = dataset[OBJETIVO]

X_train_librerias.drop(['trt','strat'], axis=1, inplace=True)
X_test_librerias.drop(['trt','strat'], axis=1, inplace=True)


Antes de hacer la comparacion, veremos primero cual es el mejor criterio a utilizar en las librerias.

In [ ]:
#Usamos el criterio entropy
ArbolDecisionLibreria_entr = DecisionTreeClassifier(criterion='entropy', random_state=12345)
ArbolDecisionLibreria_entr.fit(X_train_librerias, Y_train_librerias)
Y_predicho_arbol_libreria_entr = ArbolDecisionLibreria_entr.predict(X_test_librerias)

RandomForest_entr = RandomForestClassifier(criterion='entropy', random_state=12345)
RandomForest_entr.fit(X_train_librerias, Y_train_librerias)
Y_predicho_random_forest_entr = RandomForest_entr.predict(X_test_librerias)

# Usamos el criterio gini
ArbolDecisionLibreria_gini = DecisionTreeClassifier(criterion='gini', random_state=12345)
ArbolDecisionLibreria_gini.fit(X_train_librerias, Y_train_librerias)
Y_predicho_arbol_libreria_gini = ArbolDecisionLibreria_gini.predict(X_test_librerias)

RandomForest_gini = RandomForestClassifier(criterion='gini', random_state=12345)
RandomForest_gini.fit(X_train_librerias, Y_train_librerias)
Y_predicho_random_forest_gini = RandomForest_gini.predict(X_test_librerias)

accuracy_arbol_libreria_entr, _, _, f1_arbol_libreria_entr = funciones.get_accuracy_precision_recall_f1(Y_test_librerias, Y_predicho_arbol_libreria_entr)
accuracy_random_forest_entr, _, _, f1_random_forest_entr = funciones.get_accuracy_precision_recall_f1(Y_test_librerias, Y_predicho_random_forest_entr)


accuracy_arbol_libreria_gini, _, _, f1_arbol_libreria_gini = funciones.get_accuracy_precision_recall_f1(Y_test_librerias, Y_predicho_arbol_libreria_gini)
accuracy_random_forest_gini, _, _, f1_random_forest_gini = funciones.get_accuracy_precision_recall_f1(Y_test_librerias, Y_predicho_random_forest_gini)

print("La accuracy del arbol de decision con el criterio entropy es:", accuracy_arbol_libreria_entr)
print("La f1 del arbol de decision con el criterio entropy es:", f1_arbol_libreria_entr)
print("La accuracy del random forest con el criterio entropy es:", accuracy_random_forest_entr)
print("La f1 del random forest con el criterio entropy es:", f1_random_forest_entr)

print("\n")

print("La accuracy del arbol de decision con el criterio gini es:", accuracy_arbol_libreria_gini)
print("La f1 del arbol de decision con el criterio gini es:", f1_arbol_libreria_gini)
print("La accuracy del random forest con el criterio gini es:", accuracy_random_forest_gini)
print("La f1 del random forest con el criterio gini es:", f1_random_forest_gini)

Podemos observar como los mejores resultados son dados cuando el criterio usado en los algoritmos de `scikit-learn` es `gini`, por lo que usaremos este para las comparaciones con la linea base y nuestro algoritmo manual.

### 4.2 Resultados de la comparación

In [ ]:
accuracy_predictor_simple, _, _, f1_predictor_simple = funciones.get_accuracy_precision_recall_f1(Y_real, Y_predicho_predictor_simple)

funciones.plot_accuracies_and_f1s([accuracy_predictor_simple, accuracy_test, accuracy_arbol_libreria_gini, accuracy_random_forest_gini],
                                  [f1_predictor_simple, f1_test, f1_arbol_libreria_gini, f1_random_forest_gini])

Con la gráfica de arriba, podemos ver cómo se cumplen varias cosas:

- El predictor manual cumple con lo mínimo deseado de estar por encima de la línea base.
- El predictor manual otorga resultados competentes al nivel de la librería usada.
- RandomForest es mejor que nuestra implementación pero no por mucho.


## 5. Conclusiones

A modo de conclusión general, se logra implementar un predictor de mortalidad en pacientes con VIH utilizando árboles de decisión generados con el algoritmo ID3.

En particular, se prueban distintas configuraciones de hiper-parámetros y funciones de atributos. La combinación con mejor resultado fue utilizando la función `gain ratio`, con max_range_split de 3 y categorización _«in situ»_.

Las métricas de clasificación dieron una _accuracy_ de 0.84 y una puntuación f1 de 0.90 para la mejor configuración. Y este es competitivo con la implementación de Árboles de `scikit-learn` y, si bien el rendimiento es estrictamente inferior, es comparable con la implementación de Random Forest de la misma librería. 

Como mejoras a futuro podríamos listar:
- Explorar otros métodos de preprocesamiento de datos, como por ejemplo la selección de atributos en el dataset.
- Probar métodos más complejos como Random Forest.
- Aplicar técnicas para reducir el overfitting en caso de haberlo, como puede ser early stopping.

# Validación Cruzada

In [19]:
from sklearn import metrics
import scipy.stats

splits = 5
m_posibles = list(range(0, 1000, 10))
m_posibles[0] = 1
# Hacemos cross validation para encontrar el mejor "m"
for m in m_posibles:

    kf = KFold(n_splits=splits)
    scores = np.zeros(5)
    score_index = 0
    
    for train_index, val_index in kf.split(X_train):
        # Obtengo la nueva partición de X_train y X_val
        X_train_cv, X_val_cv= X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_cv, y_val_cv= Y_train.iloc[train_index], Y_train.iloc[val_index]

        X_train_copy = X_train_cv.copy()
        X_val_copy = X_val_cv.copy()
        
        # Calculo los nuevos puntos de cortes
        discretizer = KBinsDiscretizer(n_bins=3, encode="ordinal", strategy='kmeans', subsample=200_000, random_state=12345)
        puntos_corte = {}
        for atributo in atributos_a_categorizar:
            X_train_copy[atributo] = discretizer.fit_transform(X_train_copy[[atributo]]).astype(int)
            puntos_corte[atributo] = discretizer.bin_edges_[0][1:3]
            valores_posibles[atributo] = np.unique(X_train_copy[atributo])

        # Discretizo el resto del conjunto de datos
        for atributo in atributos_a_categorizar:
            X_val_copy[atributo] = np.digitize(X_val_copy[atributo], puntos_corte[atributo])
        
        # Defino y entreno al modelo
        modelo_cv = funciones.NaiveBayesAIDS(m, valores_posibles)
        modelo_cv.fit(X_train_copy,y_train_cv)
        y_pred = modelo_cv.predict(X_val_copy)

        # Calculo métricas
        scores[score_index] = metrics.f1_score(y_val_cv, y_pred, pos_label=0)
        score_index += 1
    print ("m: {0:d}, Accuracy media: {1:.3f} (+/-{2:.3f})".format(m, np.mean(scores), scipy.stats.sem(scores)))

m: 1, Accuracy media: 0.664 (+/-0.014)
m: 10, Accuracy media: 0.658 (+/-0.015)
m: 20, Accuracy media: 0.651 (+/-0.014)
m: 30, Accuracy media: 0.646 (+/-0.019)
m: 40, Accuracy media: 0.643 (+/-0.020)
m: 50, Accuracy media: 0.641 (+/-0.021)
m: 60, Accuracy media: 0.637 (+/-0.019)
m: 70, Accuracy media: 0.630 (+/-0.017)
m: 80, Accuracy media: 0.623 (+/-0.019)
m: 90, Accuracy media: 0.620 (+/-0.017)
m: 100, Accuracy media: 0.620 (+/-0.021)
m: 110, Accuracy media: 0.608 (+/-0.022)
m: 120, Accuracy media: 0.604 (+/-0.025)
m: 130, Accuracy media: 0.600 (+/-0.025)
m: 140, Accuracy media: 0.586 (+/-0.027)
m: 150, Accuracy media: 0.569 (+/-0.027)
m: 160, Accuracy media: 0.565 (+/-0.028)
m: 170, Accuracy media: 0.561 (+/-0.029)
m: 180, Accuracy media: 0.547 (+/-0.034)
m: 190, Accuracy media: 0.539 (+/-0.036)
m: 200, Accuracy media: 0.525 (+/-0.033)


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c37fbc8c-fee1-4291-8075-be0e33896680' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>